In [22]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier

In [23]:
df = pd.read_csv('./data/combined_data.csv')

In [24]:
df.shape


(1737, 2)

## Train Test Split

It is important that we split the data before we start training our model or create our vectorization.  When applying the model it will have no concept of the word from the original model so the data needs to be split and fit only on the training data.

In [25]:
X = df.title
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 42)

## Count Vectorization

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words
cvec = CountVectorizer(stop_words='english')

In [27]:

cvec.fit(X_train);

In [28]:
X_train = cvec.transform(X_train)
X_test = cvec.transform(X_test)

In [11]:
X_train.shape

(1302, 4872)

In [29]:
X_test.shape

(435, 4872)

### Random Forest model

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [42]:
rf = RandomForestClassifier()
rf_params = {
    'n_estimators' : [12, 14, 16],
    'max_depth'    : [10, 12, 14]
    
}

gs = GridSearchCV(rf, param_grid=rf_params)
gs.fit(X_train, y_train)
print(gs.best_score_)

0.6797235023041475


In [43]:
gs.score(X_test, y_test)

0.7011494252873564

In [44]:
gs.best_params_

{'max_depth': 14, 'n_estimators': 14}

### Gradiant Boosting with Logistic Regression